In [14]:
# Import dependecies
import pandas as pd
import numpy as np
import difflib
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
movies_df = pd.read_csv('../data/movies.csv')
movies_df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [16]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [17]:
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [18]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director', 'production_companies']

In [19]:
# Replacing null values with null string
for feature in selected_features:
    movies_df[feature] = movies_df[feature].fillna('')


In [20]:
# combining all the selected_features
combined_df = movies_df['genres'] + movies_df['keywords'] + movies_df['tagline'] + movies_df['cast'] + movies_df['director'] + movies_df['production_companies']
combined_df

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [21]:
vectorizer = TfidfVectorizer()

In [22]:
feature_vectors = vectorizer.fit_transform(combined_df)
print(feature_vectors)

  (0, 5300)	0.06658757622152417
  (0, 5506)	0.07719799906673884
  (0, 14900)	0.10006023738585378
  (0, 29328)	0.08429846010963403
  (0, 15290)	0.2323090586057348
  (0, 10029)	0.18492298683739533
  (0, 16317)	0.13828194096156365
  (0, 30556)	0.28384313631066155
  (0, 34479)	0.11065020650212594
  (0, 10280)	0.2032544217247949
  (0, 30393)	0.2323090586057348
  (0, 32012)	0.05976314733592446
  (0, 35360)	0.10525869838053162
  (0, 25214)	0.07178881654153738
  (0, 25680)	0.18084621464621511
  (0, 28984)	0.1325320943157888
  (0, 35384)	0.1991776495336147
  (0, 35803)	0.16771711203655737
  (0, 28947)	0.18280612374528368
  (0, 30031)	0.17141335003368377
  (0, 34703)	0.16659155194999575
  (0, 30947)	0.13645730924289162
  (0, 21140)	0.18974478492108335
  (0, 23447)	0.13314716087943707
  (0, 28145)	0.2323090586057348
  :	:
  (4801, 18861)	0.3167733394535455
  (4802, 24545)	0.05842367547021673
  (4802, 19078)	0.05842367547021673
  (4802, 14163)	0.06321137598988995
  (4802, 15612)	0.0587922955441139

In [23]:
# Cosine Similarity: getting the similarity confidence value
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.0683712  0.03194165 ... 0.02178793 0.         0.02108254]
 [0.0683712  1.         0.03873024 ... 0.02164503 0.         0.01241619]
 [0.03194165 0.03873024 1.         ... 0.01367098 0.         0.01068244]
 ...
 [0.02178793 0.02164503 0.01367098 ... 1.         0.         0.01118443]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.02108254 0.01241619 0.01068244 ... 0.01118443 0.         1.        ]]


In [24]:
# Getting input from user
movie_name = 'three'

In [25]:
movie_name

'three'

In [26]:
# Create a list with all the list of the movie names given in the dataset
list_of_all_titles = movies_df['title'].tolist()

In [27]:
find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_matches

['Three', 'Thirteen', 'Thr3e']

In [28]:
close_match = find_close_matches[0]
close_match

'Three'

In [29]:
idx_of_movie = movies_df[movies_df.title == close_match].index[0]
idx_of_movie

4677

In [30]:
similarity_score = list(enumerate(similarity[idx_of_movie]))
similarity_score

[(0, 0.035691034800502036),
 (1, 0.02532673988855578),
 (2, 0.018785872085557414),
 (3, 0.028762509260583127),
 (4, 0.0066552587609295015),
 (5, 0.01867460635888964),
 (6, 0.01249466619781097),
 (7, 0.01752459451312876),
 (8, 0.013494719969373509),
 (9, 0.03032022830919371),
 (10, 0.027646925490032555),
 (11, 0.006215161157942882),
 (12, 0.018121894144913026),
 (13, 0.031988793723492566),
 (14, 0.03080712593085108),
 (15, 0.0336592702082354),
 (16, 0.012250476039638716),
 (17, 0.017735573122300494),
 (18, 0.027965446513760886),
 (19, 0.031095900994611056),
 (20, 0.017483856846123427),
 (21, 0.02253305749109427),
 (22, 0.0258814290488175),
 (23, 0.030359609174370236),
 (24, 0.040552108518294154),
 (25, 0.03807227926322226),
 (26, 0.050752732551049146),
 (27, 0.0456992943434239),
 (28, 0.027718695875622062),
 (29, 0.007258698273229937),
 (30, 0.017912687934753747),
 (31, 0.01347991702635334),
 (32, 0.02861438074618014),
 (33, 0.051657010087256476),
 (34, 0.012844768035084764),
 (35, 0.03

In [37]:
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[-1], reverse=True)
sorted_similar_movies

[(4677, 0.9999999999999993),
 (4189, 0.2590879566239701),
 (301, 0.24606951237326385),
 (2277, 0.20400255432769832),
 (2724, 0.19030136002162829),
 (975, 0.17792419516902244),
 (2665, 0.17494694003107378),
 (3440, 0.16975693845674758),
 (2799, 0.16170970397419973),
 (2860, 0.15900672703579358),
 (3145, 0.13774528343672696),
 (3301, 0.13108146553235714),
 (2378, 0.1300356782832674),
 (2370, 0.12620885481981906),
 (1730, 0.12569418506020538),
 (1761, 0.12217946579367228),
 (3548, 0.11650492093199036),
 (4098, 0.09867017144512043),
 (4081, 0.09254064439167456),
 (4539, 0.08587960390154042),
 (4377, 0.08505833997997987),
 (3197, 0.07462592227815436),
 (2052, 0.07408313122315241),
 (1187, 0.06910013391883457),
 (2384, 0.06860787510961437),
 (672, 0.06836922920983969),
 (600, 0.06816372095503224),
 (1953, 0.06635872834192075),
 (2726, 0.0653913585698513),
 (490, 0.06499460552231803),
 (528, 0.06290448322267105),
 (3155, 0.06269853498556388),
 (942, 0.06259690573418474),
 (2011, 0.06240849075

In [32]:
print('Movies suggested for you: \n')

i = 1
for i in range(30):
    index = sorted_similar_movies[i][0]
    title_from_index = movies_df[movies_df.index == index]['title'].values[0]
    print(f'{i} : {title_from_index}')

Movies suggested for you: 

0 : Three
1 : Run Lola Run
2 : Cloud Atlas
3 : The Baader Meinhof Complex
4 : Downfall
5 : The International
6 : Manderlay
7 : Good bye, Lenin!
8 : Dancer in the Dark
9 : Das Boot
10 : Amour
11 : Imagine Me & You
12 : The White Ribbon
13 : Mongol: The Rise of Genghis Khan
14 : Winged Migration
15 : The NeverEnding Story
16 : Headhunters
17 : Ghost Dog: The Way of the Samurai
18 : The Lives of Others
19 : Rubber
20 : Ajami
21 : Flame & Citron
22 : Black Book
23 : Bridge of Spies
24 : Carlos
25 : Perfume: The Story of a Murderer
26 : Killer Elite
27 : Ironclad
28 : Under the Skin
29 : A Sound of Thunder


In [33]:
# Single Function
def recommend_movie(movie_name):
    list_of_all_titles = movies_df['title'].tolist()
    find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_matches[0]
    idx_of_movie = movies_df[movies_df.title == close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[idx_of_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you: \n')

    i = 1
    for i in range(30):
        index = sorted_similar_movies[i][0]
        title_from_index = movies_df[movies_df.index == index]['title'].values[0]
        print(f'{i} : {title_from_index}')


recommend_movie(movie_name = input('Enter your favorite movie name: '))
    

Movies suggested for you: 

0 : Three
1 : Run Lola Run
2 : Cloud Atlas
3 : The Baader Meinhof Complex
4 : Downfall
5 : The International
6 : Manderlay
7 : Good bye, Lenin!
8 : Dancer in the Dark
9 : Das Boot
10 : Amour
11 : Imagine Me & You
12 : The White Ribbon
13 : Mongol: The Rise of Genghis Khan
14 : Winged Migration
15 : The NeverEnding Story
16 : Headhunters
17 : Ghost Dog: The Way of the Samurai
18 : The Lives of Others
19 : Rubber
20 : Ajami
21 : Flame & Citron
22 : Black Book
23 : Bridge of Spies
24 : Carlos
25 : Perfume: The Story of a Murderer
26 : Killer Elite
27 : Ironclad
28 : Under the Skin
29 : A Sound of Thunder
